In [21]:
import pandas as pd
import spacy

In [1]:
# Charger le fichier JSON dans un dataframe
json_file_path = './commentaires_avatar.json'
df = pd.read_json(json_file_path, orient='records')

# Afficher le dataframe

df.head()

,comment
0,A great sc fi film with amazing visuals
1,The only saving grace to both Avatar is how sp...
2,Saw this twice at the movies. James Cameron is...
3,"Ofcourse I still love the original more, but t..."
4,"a not so unpredictable action sci fi flick, ba..."


In [3]:
import json

import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Étape 1 : Chargement des données
with open('./commentaires_avatar.json', 'r') as file:
    data = json.load(file)


In [7]:
comments = [item['comment'] for item in data]

# Étape 2 : Exploration des données (en option)


In [8]:

# Étape 3 : Nettoyage et tokenization
def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text

cleaned_comments = [clean_text(comment) for comment in comments]


In [9]:

# Étape 4 : Suppression des stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
filtered_comments = [' '.join([word for word in comment.split() if word not in stop_words]) for comment in cleaned_comments]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baheu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:

# Étape 5 : Vectorisation avec TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(filtered_comments).toarray()


In [11]:
# création de la colonne sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

model = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = model.polarity_scores(text)
    return scores.get('compound')
%time df['sentiment'] = np.vectorize(get_sentiment)(df.comment)

CPU times: total: 812 ms
Wall time: 1.23 s


In [29]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

def get_entities(text):
    doc = nlp(text)
    list_ = []
    for ent in doc.ents:
        if ent.label_.lower() not in ('time', 'money', 'date'):
            temp_ = re.sub('[^a-z0-9]+','_', ent.text.lower())
            list_.append('__'.join([temp_, ent.label_.upper()]))
    return ' '.join(list_)

In [ ]:
%time df['entities'] = np.vectorize(get_entities)(df.comment)

In [ ]:
df

In [37]:
df.entities.value_counts()

entities
                                                                                                                                                                                                                                          1815
first__ORDINAL                                                                                                                                                                                                                             261
3__CARDINAL                                                                                                                                                                                                                                 39
one__CARDINAL                                                                                                                                                                                                                               38
cgi__ORG                           